In [1]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google'

In [2]:
data_path = "/content/drive/MyDrive/Colab Notebooks/HealthCare_Claims.xlsx"
import pandas as pd
df = pd.read_excel(data_path)



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/HealthCare_Claims.xlsx'

In [ ]:
df.isnull().sum()


,0
encounter_id,0
patient_id,0
name,0
Gender,12666
doctor_id,0
doctor_name,0
hospital_id,0
hospital_name,0
insurance_provider_id,0
insurance_provider_name,0


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   encounter_id             12684 non-null  int64         
 1   patient_id               12684 non-null  int64         
 2   name                     12684 non-null  object        
 3   Gender                   18 non-null     object        
 4   doctor_id                12684 non-null  int64         
 5   doctor_name              12684 non-null  object        
 6   hospital_id              12684 non-null  int64         
 7   hospital_name            12684 non-null  object        
 8   insurance_provider_id    12684 non-null  int64         
 9   insurance_provider_name  12684 non-null  object        
 10  MedicalCondition         12684 non-null  object        
 11  admission_date           12684 non-null  datetime64[ns]
 12  discharge_date           12684 n

In [ ]:
df.describe()


,encounter_id,patient_id,doctor_id,hospital_id,insurance_provider_id,admission_date,discharge_date,billing_amount,Length_of_Stay
count,12684.000000,1.268400e+04,12684.000000,12684.000000,12684.000000,12684,12684,12684.000000,2.0
mean,55889.022233,1.049814e+07,19263.399558,1233.986203,16258.025386,2023-11-17 08:38:29.176915968,2023-11-25 23:01:52.393566720,57558.776180,14.0
min,19253.000000,1.000010e+07,19251.000000,1231.000000,16255.000000,2018-11-21 00:00:00,2022-10-26 00:00:00,512.520000,14.0
25%,37528.500000,1.025076e+07,19254.000000,1232.000000,16256.000000,2023-10-28 00:00:00,2023-11-04 00:00:00,28367.221250,14.0
50%,56047.500000,1.050083e+07,19261.000000,1234.000000,16258.000000,2023-11-18 00:00:00,2023-11-26 00:00:00,57582.360000,14.0
75%,74101.250000,1.074522e+07,19271.000000,1236.000000,16260.000000,2023-12-10 00:00:00,2023-12-17 00:00:00,86170.447500,14.0
max,92598.000000,1.099993e+07,19284.000000,1237.000000,16261.000000,2023-12-31 00:00:00,2024-01-19 00:00:00,614988.220000,14.0
std,21185.379706,2.869929e+05,10.177983,1.992784,2.000824,NaN,NaN,33536.232864,0.0


In [ ]:
df.fillna(0, inplace=True)


In [ ]:
df.dropna(axis=1, how='all', inplace=True)
print(df.columns)

Index(['encounter_id', 'patient_id', 'name', 'Gender', 'doctor_id',
       'doctor_name', 'hospital_id', 'hospital_name', 'insurance_provider_id',
       'insurance_provider_name', 'MedicalCondition', 'admission_date',
       'discharge_date', 'billing_amount', 'admission_type', 'medication',
       'test_resultS', 'Length_of_Stay'],
      dtype='object')


In [1]:
!pip install streamlit pyngrok pandas numpy matplotlib -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.2 MB/s eta 0:00:00


In [15]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ====================================================================
# Prediction and Explanation Simulation Function (For Demo Purposes)
# ====================================================================

def get_prediction_and_explanation(claim_amount):
    """Simulates risk prediction and generates SHAP values to represent evidence."""

    # 1. Simulate Risk Score
    # The risk score increases with claim amount, plus some randomness
    risk_score = np.clip(0.1 + (claim_amount / 10000) * np.random.uniform(0.5, 0.8), 0.1, 0.99)
    prediction = "High Risk" if risk_score > 0.65 else "Low Risk"

    # 2. Simulate SHAP values (Evidence)
    features = [
        'Provider Cost Deviation',
        'Procedure/Patient Frequency',
        'Diagnosis Inconsistency',
        'Total Claim Amount',
        'Prior Provider History'
    ]
    shap_values = np.random.uniform(-0.4, 0.4, len(features))

    # Define factors that raise or lower risk based on the result
    if prediction == "High Risk":
        # Make the top factors strongly positive for High Risk
        shap_values[0] = np.random.uniform(0.3, 0.5)
        shap_values[1] = np.random.uniform(0.2, 0.4)
        shap_values[3] = risk_score * 0.1
    else:
        # Make some factors strongly negative for Low Risk
        shap_values[2] = np.random.uniform(-0.3, -0.5)
        shap_values[4] = np.random.uniform(-0.2, -0.4)

    explanation_df = pd.DataFrame({
        'Factor': features,
        'Risk_Impact': shap_values
    }).sort_values(by='Risk_Impact', ascending=False)

    return prediction, risk_score, explanation_df

# ====================================================================
# Streamlit Interface Design
# ====================================================================

st.set_page_config(page_title="Hissn - Medical Fraud Detection", layout="wide")

st.title("🛡️ Hissn: Smart Fraud, Waste & Abuse (FWA) Detection - Prototype")
st.markdown("---")

col1, col2 = st.columns([1, 2])

with col1:
    st.header("Claim Input for Analysis")
    with st.form("claims_input_form"):
        claim_id = st.text_input("Claim ID", value="C1234567")
        provider_id = st.text_input("Provider ID (Doctor ID)", value="D9876")
        claim_amount = st.number_input("Total Claim Amount (SAR)", value=5500.00, min_value=1.0)

        submitted = st.form_submit_button("Analyze Claim 🚀")

with col2:
    if submitted:
        # 1. Get Prediction and Evidence
        prediction, risk_score, explanation_df = get_prediction_and_explanation(claim_amount)

        # 2. Display Result with st.metric and improved visuals
        risk_percentage = f"{risk_score * 100:.1f}%"

        # Display Risk Probability prominently using st.metric
        st.metric(label="Estimated Fraud Probability", value=risk_percentage)

        if prediction == "High Risk":
            # Enhanced visual alert for High Risk
            st.markdown(f"<p style='padding: 10px; border-radius: 5px; background-color: #fce4e4; color: #b71c1c; font-size: 1.2em; font-weight: bold;'>⚠️ HIGH RISK CLAIM: Requires Immediate Review</p>", unsafe_allow_html=True)
            st.markdown(f"Risk Classification: **{prediction}**")
            st.markdown("---")
            st.error("🚨 System Recommendation: Freeze claim and initiate immediate audit based on the evidence below.")
        else:
            # Enhanced visual confirmation for Low Risk
            st.markdown(f"<p style='padding: 10px; border-radius: 5px; background-color: #e8f5e9; color: #2e7d32; font-size: 1.2em; font-weight: bold;'>✅ LOW RISK CLAIM: Safe to Process</p>", unsafe_allow_html=True)
            st.markdown(f"Risk Classification: **{prediction}**")
            st.markdown("---")
            st.info("👍 System Recommendation: Proceed with automated processing.")

        # 3. Display Evidence (SHAP Explanation)
        st.header("SHAP-Supported Evidence (Why this Decision?)")

        # Improved Matplotlib chart aesthetics
        plt.style.use('ggplot')
        fig, ax = plt.subplots(figsize=(10, len(explanation_df) * 0.5 + 1))
        # Color positive impacts red (increasing risk) and negative impacts blue (decreasing risk)
        colors = ['#EF5350' if val > 0 else '#42A5F5' for val in explanation_df['Risk_Impact']]

        ax.barh(explanation_df['Factor'], explanation_df['Risk_Impact'], color=colors)
        ax.set_xlabel("Factor Impact on Risk Decision (SHAP Value)", fontsize=12)
        ax.set_title(f"Top {len(explanation_df)} Factors Influencing Prediction", fontsize=14, pad=15)
        ax.axvline(0, color='gray', linewidth=0.8, linestyle='--') # Baseline at zero
        ax.invert_yaxis() # Display highest impact factors at the top
        ax.tick_params(axis='both', which='major', labelsize=10)

        st.pyplot(fig)


Overwriting app.py


In [16]:
from pyngrok import ngrok
import subprocess
import time # Import the time module

# --- NGROK CONFIGURATION ---
# The confirmed valid Authtoken for ngrok.
NGROK_AUTH_TOKEN = "34PYa4sBH4g0rYaHhbiHENj7sCi_LzmVmmukeWyEQ4Jx3xdp"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Define the Streamlit port
PORT = 8501

# Kill any previous ngrok processes to ensure a clean start
# This step is crucial to avoid the ERR_NGROK_108 limit (1 simultaneous session).
ngrok.kill()
time.sleep(1) # Added a small delay to ensure process termination completes

# --- STREAMLIT EXECUTION ---
# Run the 'app.py' Streamlit file in the background
process = subprocess.Popen([
    "streamlit",
    "run",
    "app.py",
    "--server.port",
    str(PORT),
    "--server.enableCORS",
    "false"
])

# --- NGROK TUNNEL CREATION ---
# Open an ngrok tunnel to the Streamlit port (this will now authenticate successfully)
public_url = ngrok.connect(PORT)

# --- OUTPUT ---
print(f"=====================================================")
print(f"🚀 Streamlit app is ready! Open the following URL in your browser:")
print(public_url)
print(f"=====================================================")


🚀 Streamlit app is ready! Open the following URL in your browser:
NgrokTunnel: "https://sepalled-sarina-hairless.ngrok-free.dev" -> "http://localhost:8501"
